In [1]:
!pip install --quiet arcgis

In [2]:
import os
from arcgis.gis import GIS;
import pandas as pd;
import requests;

In [3]:
pd_orgid = pd.DataFrame(columns = [
     "stateID"
    ,"organizationIdentifier"
]);

pd_assmnts = pd.DataFrame(columns = [
     "organizationIdentifier"
    ,"assessmentUnitIdentifier"
    ,"reportingCycle"
    ,"agencyCode"
    ,"trophicStatusCode"
    ,"epaIRCategory"
]);

pd_uses = pd.DataFrame(columns = [
     "organizationIdentifier"
    ,"assessmentUnitIdentifier"
    ,"reportingCycle"
    ,"useName"
    ,"useAttainmentCode"
    ,"threatenedIndicator"
    ,"trendCode"
    ,"agencyCode"
    ,"assessmentMetadata"
]);

pd_parms = pd.DataFrame(columns = [
     "organizationIdentifier"
    ,"assessmentUnitIdentifier"
    ,"reportingCycle"
    ,"parameterStatusName"
    ,"parameterName"
    ,"pollutantIndicator"
]);

pd_parms2use = pd.DataFrame(columns = [
     "organizationIdentifier"
    ,"assessmentUnitIdentifier"
    ,"reportingCycle"
    ,"parameterName"
    ,"associatedUseName"
    ,"parameterAttainmentCode"
    ,"trendCode"
]);

pd_srcs = pd.DataFrame(columns = [
     "organizationIdentifier"
    ,"assessmentUnitIdentifier"
    ,"reportingCycle"
    ,"sourceName"
    ,"sourceConfirmedIndicator"
]);

pd_src2caus = pd.DataFrame(columns = [
     "organizationIdentifier"
    ,"assessmentUnitIdentifier"
    ,"reportingCycle"
    ,"sourceName"
    ,"causeName"
]);

pd_docs = pd.DataFrame(columns = [
     "organizationIdentifier"
    ,"assessmentUnitIdentifier"
    ,"reportingCycle"
    ,"documentName"
    ,"documentFileName"
    ,"documentURL"
    ,"agencyCode"
]);


In [4]:
# Pull the list of organization identifiers
org_url = r"https://attains.epa.gov/attains-public/api/domains";
response = requests.get(
     org_url
    ,params={
         'domainName': 'OrgStateCode'
    }
);
resp_json = response.json();

orgs = [];
for item in resp_json:
    
    if item['context'] == "EPA" or item['context'][:5] == "TEST_":
        continue;

    orgs.append({
         "stateID": item['code']
        ,"organizationIdentifier": item['context']
    });
    
    
pd_orgid = pd_orgid.head(0)                                 \
    .append(orgs)                                           \
    .sort_values(by=['stateID','organizationIdentifier']);
    
len(pd_orgid)

71

In [6]:
# For each orgid pull the assessments into a table
ste_url = r"https://attains.epa.gov/attains-public/api/assessments"

for index, row in pd_orgid.iterrows():
    
    response = requests.get(
         ste_url
        ,params = {
             "organizationId": row['organizationIdentifier']
        }
    );
    resp_json = response.json();
    print(resp_json['items'][0])
    cycle_year = resp_json['items'][0]['reportingCycleText'];
    
    assmnt   = [];
    uses     = [];
    parms    = [];
    parm2use = [];
    srcs     = [];
    src2caus = [];
    docs     = [];
    
    for item in resp_json['items'][0]['assessments']:
        assmnt.append({
             "organizationIdentifier": row['organizationIdentifier']
            ,"assessmentUnitIdentifier": item['assessmentUnitIdentifier']
            ,'reportingCycle': cycle_year
            ,"agencyCode": item['agencyCode']
            ,"trophicStatusCode": item['trophicStatusCode']
            ,"epaIRCategory": item['epaIRCategory']
        });
        
        if item['useAttainments'] is not None and len(item['useAttainments']) > 0:
            for uitem in item['useAttainments']:
                uses.append({
                     "organizationIdentifier": row['organizationIdentifier']
                    ,"assessmentUnitIdentifier": item['assessmentUnitIdentifier']
                    ,"reportingCycle": cycle_year
                    ,"useName": uitem['useName']
                    ,"useAttainmentCode": uitem['useAttainmentCode']
                    ,"threatenedIndicator": uitem['threatenedIndicator']
                    ,"trendCode": uitem['trendCode']
                    ,"agencyCode": uitem['agencyCode']
                    ,"assessmentMetadata": uitem['assessmentMetadata']
                });
                
        if item['parameters'] is not None and len(item['parameters']) > 0:
            for pitem in item['parameters']:
                parms.append({
                     "organizationIdentifier": row['organizationIdentifier']
                    ,"assessmentUnitIdentifier": item['assessmentUnitIdentifier']
                    ,"reportingCycle": cycle_year
                    ,"parameterStatusName": pitem['parameterStatusName']
                    ,"parameterName": pitem['parameterName']
                    ,"pollutantIndicator": pitem['pollutantIndicator']
                });
                
                if pitem['associatedUses'] is not None and len(pitem['associatedUses']) > 0:
                    for auitem in pitem['associatedUses']:
                        parm2use.append({
                             "organizationIdentifier": row['organizationIdentifier']
                            ,"assessmentUnitIdentifier": item['assessmentUnitIdentifier']
                            ,"reportingCycle": cycle_year
                            ,"parameterName": pitem['parameterName']
                            ,"associatedUseName": auitem['associatedUseName']
                            ,"parameterAttainmentCode": auitem['parameterAttainmentCode']
                            ,"trendCode": auitem['trendCode']
                        });
                        
        if item['probableSources'] is not None and len(item['probableSources']) > 0:
            for psitem in item['probableSources']:
                srcs.append({
                     "organizationIdentifier": row['organizationIdentifier']
                    ,"assessmentUnitIdentifier": item['assessmentUnitIdentifier']
                    ,"reportingCycle": cycle_year
                    ,"sourceName": psitem['sourceName']
                    ,"sourceConfirmedIndicator": psitem['sourceConfirmedIndicator']
                });
                
                if psitem['associatedCauseNames'] is not None and len(psitem['associatedCauseNames']) > 0:
                    for acitem in psitem['associatedCauseNames']:
                        src2caus.append({
                             "organizationIdentifier": row['organizationIdentifier']
                            ,"assessmentUnitIdentifier": item['assessmentUnitIdentifier']
                            ,"reportingCycle": cycle_year
                            ,"sourceName": psitem['sourceName']
                            ,"causeName": acitem['causeName']
                        });
                        
        if item['documents'] is not None and len(item['documents']) > 0:
            for docitem in item['documents']:
                docs.append({
                     "organizationIdentifier": row['organizationIdentifier']
                    ,"assessmentUnitIdentifier": item['assessmentUnitIdentifier']
                    ,"reportingCycle": cycle_year
                    ,"documentName": docitem['documentName']
                    ,"documentFileName": docitem['documentFileName']
                    ,"documentURL": docitem['documentURL']
                    ,"agencyCode": docitem['agencyCode']
                });
                
    
        if index > 1:
            break
            
        pd_assmnts = pd_assmnts.append(assmnt)

pd_assmnts

{'organizationIdentifier': 'AKDECWQ', 'organizationName': 'Alaska', 'organizationTypeText': 'State', 'reportingCycleText': '2016', 'combinedCycles': [{'combinedCycleText': '2014'}], 'reportStatusCode': 'EPA Final Action', 'documents': [{'agencyCode': 'S', 'documentTypes': [{'documentTypeCode': 'Other Supporting Documents'}], 'documentFileType': 'text/html', 'documentFileName': 'High Priority Waters to be Addressed in 2016.docx', 'documentName': 'High Priority Waters to be Addressed in 2016', 'documentDescription': None, 'documentComments': 'This document is taken from Table 2 in the Alaska Nonpoint Source Water Pollution Control Strategy :Fiscal Years 2016-2020 - http://dec.alaska.gov/water/wqsar/Docs/AlaskaNonpointSourceWaterPollutionControlStrategy.pdf', 'documentURL': 'https://attains.epa.gov/attains-public/api/documents/cycles/3214/133148'}, {'agencyCode': 'S', 'documentTypes': [{'documentTypeCode': 'Integrated Report Document'}], 'documentFileType': 'text/html', 'documentFileName'

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{'organizationIdentifier': '21AS', 'organizationName': 'American Samoa', 'organizationTypeText': 'State', 'reportingCycleText': '2018', 'combinedCycles': [], 'reportStatusCode': 'EPA Final Action', 'documents': [{'agencyCode': 'E', 'documentTypes': [{'documentTypeCode': 'EPA Decision Rationale'}], 'documentFileType': 'text/html', 'documentFileName': 'American Samoa 2018 IR Approval Enclosure-052219.pdf', 'documentName': 'EPA approval letter enclosure for American Samoa 2018 303d list dated 5/22/2019', 'documentDescription': None, 'documentComments': None, 'documentURL': 'https://attains.epa.gov/attains-public/api/documents/cycles/3182/134381'}, {'agencyCode': 'S', 'documentTypes': [{'documentTypeCode': 'Appendices'}, {'documentTypeCode': 'Integrated Report Document'}], 'documentFileType': 'text/html', 'documentFileName': '2018 American Samoa Integrated Report .pdf', 'documentName': '2018 Integrated Report and Appendices', 'documentDescription': None, 'documentComments': None, 'document

{'organizationIdentifier': '21ARIZ', 'organizationName': 'Arizona', 'organizationTypeText': 'State', 'reportingCycleText': '2018', 'combinedCycles': [], 'reportStatusCode': 'EPA Final Action', 'documents': [{'agencyCode': 'E', 'documentTypes': [{'documentTypeCode': 'EPA Transmittal Letter'}], 'documentFileType': 'text/html', 'documentFileName': 'AZ 2018 303(d) List_EPA Final Letter.pdf', 'documentName': "EPA Letter Concerning Final Action on Arizona's 2018 303(d) List", 'documentDescription': None, 'documentComments': None, 'documentURL': 'https://attains.epa.gov/attains-public/api/documents/cycles/1903/137121'}, {'agencyCode': 'E', 'documentTypes': [{'documentTypeCode': 'EPA Decision Rationale'}], 'documentFileType': 'text/html', 'documentFileName': 'AZ 2018 303(d) List_EPA Final Letter Enclosure.pdf', 'documentName': "EPA Letter Enclosure Concerning Final Action on Arizona's 2018 303(d) List", 'documentDescription': None, 'documentComments': None, 'documentURL': 'https://attains.epa.

IndexError: list index out of range